In [23]:
import subprocess
import os

import pandas as pd

import requests
from bs4 import BeautifulSoup

import json


pd.set_option('display.max_rows', 500)

* RKI, webscrape (webscraping)           https://www.rki.de/DE/Content/InfAZ/N/Neuartiges_Coronavirus/Fallzahlen.html
* John Hopkins (GITHUB) https://github.com/CSSEGISandData/COVID-19
* REST API services to retreive data https://npgeo-corona-npgeo-de.hub.arcgis.com/

In [24]:

git_pull = subprocess.Popen( "/usr/bin/git pull" , 
                     cwd = os.path.dirname( 'C:/Users/aaris/ads_covid-19/data/raw/COVID-19/' ), 
                     shell = True, 
                     stdout = subprocess.PIPE, 
                     stderr = subprocess.PIPE )
(out, error) = git_pull.communicate()


print("Error : " + str(error)) 
print("out : " + str(out))

Error : b'The system cannot find the path specified.\r\n'
out : b''


In [25]:
data_path='../data/raw/COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'
pd_raw=pd.read_csv(data_path)

In [26]:
pd_raw

Province/State                    Country/Region  \
0                                 NaN                       Afghanistan   
1                                 NaN                           Albania   
2                                 NaN                           Algeria   
3                                 NaN                           Andorra   
4                                 NaN                            Angola   
5                                 NaN               Antigua and Barbuda   
6                                 NaN                         Argentina   
7                                 NaN                           Armenia   
8        Australian Capital Territory                         Australia   
9                     New South Wales                         Australia   
10                 Northern Territory                         Australia   
11                         Queensland                         Australia   
12                    South Australia                         Australia   
13                           Tasmania                         Australia   
14                           Victoria                         Australia   
15                  Western Australia                         Australia   
16                                NaN                           Austria   
17                                NaN                        Azerbaijan   
18                                NaN                           Bahamas   
19                                NaN                           Bahrain   
20                                NaN                        Bangladesh   
21                                NaN                          Barbados   
22                                NaN                           Belarus   
23                                NaN                           Belgium   
24                                NaN                            Belize   
25                                NaN                             Benin   
26                                NaN                            Bhutan   
27                                NaN                           Bolivia   
28                                NaN            Bosnia and Herzegovina   
29                                NaN                          Botswana   
30                                NaN                            Brazil   
31                                NaN                            Brunei   
32                                NaN                          Bulgaria   
33                                NaN                      Burkina Faso   
34                                NaN                             Burma   
35                                NaN                           Burundi   
36                                NaN                        Cabo Verde   
37                                NaN                          Cambodia   
38                                NaN                          Cameroon   
39                            Alberta                            Canada   
40                   British Columbia                            Canada   
41                   Diamond Princess                            Canada   
42                     Grand Princess                            Canada   
43                           Manitoba                            Canada   
44                      New Brunswick                            Canada   
45          Newfoundland and Labrador                            Canada   
46              Northwest Territories                            Canada   
47                        Nova Scotia                            Canada   
48                            Ontario                            Canada   
49               Prince Edward Island                            Canada   
50                             Quebec                            Canada   
51                       Saskatchewan                            Canada   
52                              Yukon           

Webscrapping

In [27]:
page = requests.get("https://www.rki.de/DE/Content/InfAZ/N/Neuartiges_Coronavirus/Fallzahlen.html")

In [28]:
soup = BeautifulSoup(page.content, 'html.parser')

In [29]:
html_table=soup.find('table')

In [30]:
all_rows=html_table.find_all('tr')

In [31]:
final_data_list=[]

In [32]:
for pos,rows in enumerate(all_rows):
   
    col_list=[each_col.get_text(strip=True) for each_col in rows.find_all('td')]
    final_data_list.append(col_list)

In [33]:
pd.DataFrame(final_data_list).dropna().rename(columns={0:'state',
                                                       1:'cases',
                                                       2:'changes',
                                                       3:'cases_per_100k',
                                                       4:'fatal',
                                                       5:'comment'})

state    cases changes cases_per_100k fatal comment
2         Baden-Württem­berg   44.903    +285          1.440  13,0   1.867
3                     Bayern   61.561    +403          2.415  18,5   2.645
4                     Berlin   12.216    +126            529  14,1     226
5               Branden­burg    4.003     +19             66   2,6     169
6                     Bremen    2.117      +8             61   8,9      58
7                    Hamburg    6.770     +71            266  14,4     267
8                     Hessen   16.714    +111            663  10,6     538
9   Meck­lenburg-Vor­pommern    1.055      +9             30   1,9      20
10            Nieder­sachsen   17.783    +138            524   6,6     667
11     Nord­rhein-West­falen   61.820    +315          1.619   9,0   1.828
12          Rhein­land-Pfalz    9.649     +26            323   7,9     246
13                  Saarland    3.225      +3             32   3,2     175
14                   Sachsen    6.351     +69            202   5,0     226
15            Sachsen-Anhalt    2.330     +16             49   2,2      66
16       Schles­wig-Holstein    4.218     +11             99   3,4     161
17                 Thüringen    3.765     +20             79   3,7     188
18                    Gesamt  258.480  +1.630          8.397  10,1   9.347

API

In [34]:
data=requests.get('https://services7.arcgis.com/mOBPykOjAyBO2ZKk/arcgis/rest/services/Coronaf%C3%A4lle_in_den_Bundesl%C3%A4ndern/FeatureServer/0/query?where=1%3D1&outFields=*&outSR=4326&f=json')

In [35]:
json_object=json.loads(data.content) 

In [36]:
type(json_object)

dict

In [37]:
json_object.keys()

dict_keys(['objectIdFieldName', 'uniqueIdField', 'globalIdFieldName', 'geometryProperties', 'serverGens', 'geometryType', 'spatialReference', 'fields', 'features'])

In [38]:
full_list=[]
for pos,each_dict in enumerate (json_object['features'][:]):
    full_list.append(each_dict['attributes'])

In [39]:
pd_full_list=pd.DataFrame(full_list)
pd_full_list.head()

AGS_TXT  Aktualisierung  Death  Fallzahl  \
0      01   1599861600000    161      4218   
1      02   1599861600000    267      6770   
2      03   1599861600000    667     17783   
3      04   1599861600000     58      2117   
4      05   1599861600000   1828     61820   

                               GlobalID LAN_ew_AGS            LAN_ew_BEZ  \
0  fc5ba936-c95c-432c-8a33-9eb2f30b660f         01                  Land   
1  0f3e860c-5181-4d3f-a421-1d51f50315ea         02  Freie und Hansestadt   
2  3fd77024-c29b-4843-9be8-682ad48e60c9         03                  Land   
3  4132268b-54de-4327-ac1e-760e915112f1         04      Freie Hansestadt   
4  561d658f-3ee5-46e3-bc95-3528c6558ab9         05                  Land   

   LAN_ew_EWZ           LAN_ew_GEN  OBJECTID  OBJECTID_1   Shape__Area  \
0     2896712   Schleswig-Holstein        15           1  4.573731e+10   
1     1841179              Hamburg         6           2  2.089396e+09   
2     7982448        Niedersachsen         9           3  1.299836e+11   
3      682986               Bremen         5           4  1.119157e+09   
4    17932651  Nordrhein-Westfalen        10           5  8.782936e+10   

   Shape__Length  faelle_100000_EW  
0   2.881496e+06        145.613371  
1   4.188002e+05        367.699175  
2   4.008988e+06        222.776271  
3   3.357177e+05        309.962430  
4   2.648673e+06        344.734306

API access via REST service, e.g. USA data
example of a REST conform interface (attention registration mandatory)

www.smartable.ai

In [40]:
headers = {
    'Cache-Control': 'no-cache',
    'Subscription-Key': '28ee4219700f48718be78b057beb7eb4',
}

response = requests.get('https://api.smartable.ai/coronavirus/stats/US', headers=headers)

ConnectionError: HTTPSConnectionPool(host='api.smartable.ai', port=443): Max retries exceeded with url: /coronavirus/stats/US (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x0000019C36A12438>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))

In [ ]:
print(response)

In [41]:
US_dict=json.loads(response.content) 
with open('C:/Users/aaris/ads_covid-19/data/raw/US_data.txt', 'w') as outfile:
    json.dump(US_dict, outfile,indent=2)

NameError: name 'response' is not defined